# 04 — LLM Formatter (strict JSON contract)
**Date:** 2025-08-09

Keep models out of your data logic. Use the LLM only to format or explain.


In [ ]:
#r "nuget: Microsoft.Recognizers.Text, 1.8.13"
#r "nuget: Microsoft.Recognizers.Text.DateTime, 1.8.13"
#r "nuget: FuzzySharp, 2.0.2"
#r "nuget: RestSharp, 112.1.0"
#r "nuget: Microsoft.Extensions.Caching.Memory, 9.0.8"
#r "nuget: System.Text.Json, 9.0.0"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.Json;
using Microsoft.Recognizers.Text;
using Microsoft.Recognizers.Text.DateTime;
using FuzzySharp;
using RestSharp;

public enum Intent { GetContactInfo, FilterByHireDate, FilterByRole, Unknown }

public record Slots(
    string[]? Names = null,
    DateTime? Date = null,
    (DateTime Start, DateTime End)? Range = null,
    string? Operator = null,
    string? Department = null,
    string? Role = null
);

public record QuerySpec(Intent Intent, Slots Slots);

public record Employee(
    string DisplayName,
    string Email,
    string Department,
    string Role,
    DateTime OriginalHireDate
);

var employees = new List<Employee> {
    new("Rick Sanchez",   "rick.sanchez@company.com",   "Engineering", "Staff Engineer",  new DateTime(2015,  5, 10)),
    new("Morty Smith",    "morty.smith@company.com",    "Engineering", "Engineer I",      new DateTime(2023, 10, 12)),
    new("Summer Smith",   "summer.smith@company.com",   "Product",     "PM",              new DateTime(2021,  2,  1)),
    new("Beth Smith",     "beth.smith@company.com",     "HR",          "HR Manager",      new DateTime(2019,  7,  3)),
    new("Jerry Smith",    "jerry.smith@company.com",    "Sales",       "Account Manager", new DateTime(2022,  9, 15))
};

Console.WriteLine($"Loaded demo employees: {employees.Count}");


In [ ]:
bool enableCall = false; // flip when ready

public record AnswerDto(List<PersonDto> Answer);
public record PersonDto(string Name, string Email);

static string BuildEmailFormattingPrompt(IEnumerable<Employee> people, string question)
{
    var context = people.Select(p => new { name = p.DisplayName, email = p.Email });
    var ctxJson = JsonSerializer.Serialize(context);
    return $@"
You are an API that returns ONLY compact JSON matching this schema:
{{ ""answer"": [ {{ ""name"": ""string"", ""email"": ""string"" }} ] }}
Do not include markdown fences, commentary, or extra keys.

Question: {question}
Context (JSON array of people): {ctxJson}
Return ONLY JSON.
";
}

var question = "What are Rick and Summer's emails?";
var selected = employees.Where(e => e.DisplayName.Contains("Rick") || e.DisplayName.Contains("Summer"));

if (enableCall)
{
    var client = new RestClient("http://localhost:11434");
    var req = new RestRequest("/api/generate").AddJsonBody(new {
        model = "phi3:mini",
        prompt = BuildEmailFormattingPrompt(selected, question),
        stream = false
    });
    var resp = await client.PostAsync(req);
    var raw = resp.Content ?? "{}";

    using var doc = JsonDocument.Parse(raw);
    var text = doc.RootElement.GetProperty("response").GetString();

    try
    {
        var parsed = JsonSerializer.Deserialize<AnswerDto>(text!, new JsonSerializerOptions { PropertyNameCaseInsensitive = true });
        Console.WriteLine("LLM JSON parsed:");
        foreach (var p in parsed!.Answer) Console.WriteLine($"{p.Name}: {p.Email}");
    }
    catch (Exception ex)
    {
        Console.WriteLine("Failed to parse LLM JSON. Raw text follows:");
        Console.WriteLine(text);
        Console.WriteLine(ex.Message);
    }
}
else
{
    Console.WriteLine("Ollama call disabled. Rendering locally:");
    foreach (var p in selected) Console.WriteLine($"{p.DisplayName}: {p.Email}");
}
